1. Clone CLEAN (v1.0.1) from GitHub

In [ ]:
!git clone https://github.com/tttianhao/CLEAN.git

fatal: destination path 'CLEAN' already exists and is not an empty directory.


2. In Colab, change Directory to /content/CLEAN/app
Then you could see:
['gmm.py',
 'train-supconH.py',
 'requirements.txt',
 'results',
 'build.py',
 'data',
 'DEMO.ipynb',
 'CLEAN_infer_fasta.py',
 'src',
 'inference.py',
 'train-triplet.py']

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/MUTarget_data/combined_columns.csv', '/content/CLEAN/app/data/combined_columns.csv')
shutil.copy('/content/drive/MyDrive/MUTarget_data/new.csv', '/content/CLEAN/app/data/new.csv')
import os
os.chdir('/content/CLEAN/app')
current_path = os.getcwd()
os.listdir(current_path)

['requirements.txt',
 'CLEAN_infer_fasta.py',
 'src',
 'data',
 'DEMO.ipynb',
 'gmm.py',
 'inference.py',
 'train-supconH.py',
 'train-triplet.py',
 'esm',
 'build.py',
 'results']

3. Install the latest release of esm, clone esm gits from GitHub

In [ ]:
!pip install fair-esm  # latest release
!git clone https://github.com/facebookresearch/esm.git

fatal: destination path 'esm' already exists and is not an empty directory.


4. Retrive esm from dataset, upload your dataset to ./data

In [ ]:
file_path = 'src/CLEAN/utils.py'
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace('esm1b_t33_650M_UR50S', 'esm2_t6_8M_UR50D')

In [ ]:
# from src.CLEAN.utils import csv_to_fasta, mutate_single_seq_ECs, retrive_esm1b_embedding, compute_esm_distance
# csv_to_fasta("data/combined_columns.csv", "data/combined_columns.fasta")
# retrive_esm1b_embedding("combined_columns")

5. Mutate single sequences and retrive esm from the mutated sequences

In [ ]:
# from src.CLEAN.utils import mutate_single_seq_ECs, retrive_esm1b_embedding
# train_file = "combined_columns"
# train_fasta_file = mutate_single_seq_ECs(train_file)
# train_fasta_file

In [ ]:
# retrive_esm1b_embedding(train_fasta_file)

6. Compute default esm distance for training

In [ ]:
# from src.CLEAN.utils import compute_esm_distance
# train_file = "combined_columns"
# compute_esm_distance(train_file)

In [ ]:
file_path = 'train-triplet.py'
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace('from CLEAN.', 'from src.CLEAN.')
with open(file_path, 'w') as file:
    file.write(content)
with open('data/model/triplettest_10.pth', 'w') as file:
    file.write('')
!python train-triplet.py --training_data combined_columns --model_name triplettest --epoch 10

==> device used: cuda:0 | dtype used:  torch.float32 
==> args: Namespace(learning_rate=0.0005, epoch=10, model_name='triplettest', training_data='combined_columns', hidden_dim=512, out_dim=128, adaptive_rate=100, verbose=False)
The number of unique EC numbers:  9
---------------------------------------------------------------------------
| end of epoch   1 | time:  0.19s | training loss 0.9969
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   2 | time:  0.01s | training loss 1.0215
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   3 | time:  0.01s | training loss 0.8902
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   4 | time:  0.02

In [ ]:
file_path = 'train-supconH.py'
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace('from CLEAN.', 'from src.CLEAN.')
with open(file_path, 'w') as file:
    file.write(content)
with open('data/model/supcontest_10.pth', 'w') as file:
    file.write('')
!python train-supconH.py --training_data combined_columns --model_name supcontest --epoch 10 --n_pos 9 --n_neg 30 -T 0.1

==> device used: cuda:0 | dtype used:  torch.float32 
==> args: Namespace(learning_rate=0.0005, epoch=10, model_name='supcontest', training_data='combined_columns', temp=0.1, n_pos=9, n_neg=30, hidden_dim=512, out_dim=256, adaptive_rate=60, verbose=False)
The number of unique EC numbers:  9
---------------------------------------------------------------------------
| end of epoch   1 | time:  0.29s | training loss 3.6959
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   2 | time:  0.09s | training loss 3.6290
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   3 | time:  0.08s | training loss 3.6345
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end

In [ ]:
from src.CLEAN.utils import csv_to_fasta, mutate_single_seq_ECs, retrive_esm1b_embedding, compute_esm_distance
csv_to_fasta("data/new.csv", "data/new.fasta")
retrive_esm1b_embedding("new")

In [ ]:
from src.CLEAN.infer import infer_pvalue
train_data = "combined_columns"
test_data = "new"
with open('src/CLEAN/infer.py', 'r') as file:
    content = file.read()
content = content.replace('LayerNormNet(512, 128', 'LayerNormNet(512, 256')

In [ ]:
infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20, report_metrics=True, pretrained=False, model_name='supcontest')

The embedding sizes for train and test: torch.Size([5398, 256]) torch.Size([10, 256])


100%|██████████| 9/9 [00:00<00:00, 1265.46it/s]


Calculating eval distance map, between 10 test ids and 9 train EC cluster centers


10it [00:00, 335.40it/s]
100%|██████████| 9/9 [00:00<00:00, 3454.95it/s]
20000it [00:01, 10225.62it/s]
100%|██████████| 10/10 [00:00<00:00, 221.62it/s]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
from src.CLEAN.infer import infer_pvalue
train_data = "combined_columns"
test_data = "new"
with open('src/CLEAN/infer.py', 'r') as file:
    content = file.read()
content = content.replace('LayerNormNet(512, 128', 'LayerNormNet(512, 256')

In [ ]:
infer_pvalue(train_data, test_data, p_value=1e-5, nk_random=20, report_metrics=True, pretrained=False, model_name='triplettest')

RuntimeError: Error(s) in loading state_dict for LayerNormNet:
	size mismatch for fc3.weight: copying a param with shape torch.Size([128, 512]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).